In [1]:
#import modules and data, drop derived fields
import pandas as pd
import numpy as np
noaa = pd.read_csv(r"hourly_data.csv")



C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17,18,19,20,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#NA precipitation values are filled with zeros, as per description in data dictionary
noaa['HOURLYPrecip'] = noaa['HOURLYPrecip'].fillna(0)

In [3]:
#trace values are replaced with zeros; asterisks (which mark extreme values) removed from temperatures
noaa['HOURLYPrecip'] = noaa['HOURLYPrecip'].str.replace('T','0')
noaa['HOURLYDRYBULBTEMPF'] = noaa['HOURLYDRYBULBTEMPF'].str.replace('*','')

In [4]:
#suspect values removed from dataset
noaa = noaa[~noaa.HOURLYPrecip.str.contains("s", na=False)]
noaa = noaa[~noaa.HOURLYDRYBULBTEMPF.str.contains("s", na=False)]

In [5]:
#convert relevant values to numeric
noaa.HOURLYPrecip = pd.to_numeric(noaa.HOURLYPrecip)
noaa.HOURLYDRYBULBTEMPF = pd.to_numeric(noaa.HOURLYDRYBULBTEMPF)

#filter out stations with less than 95% data coverage for airtemp and rainonoff
goodstations = []
okstations = []
for num in noaa.groupby('STATION'):
    if ((noaa[noaa['STATION'] == num[0]].HOURLYDRYBULBTEMPF.isna().sum()/len(noaa[noaa['STATION'] == num[0]]))) <= .05:
        goodstations.append(num[0])
    if ((noaa[noaa['STATION'] == num[0]].HOURLYDRYBULBTEMPF.isna().sum()/len(noaa[noaa['STATION'] == num[0]]))) <= .2 and ((noaa[noaa['STATION'] == num[0]].HOURLYDRYBULBTEMPF.isna().sum()/len(noaa[noaa['STATION'] == num[0]]))) > .05:
        okstations.append(num[0])
        
#all 55 stations included in the dataset have pretty good data coverage
        




In [6]:
#determine average hourly precipitation rate for vdot observations where rainonoff = on
#vdot[vdot['rainonoff'] == 1].rainperhr.mean()


In [7]:
#convert time column to datetime and set to dataframe index (datetime index)
noaa.DATE = pd.to_datetime(noaa.DATE)
noaa = noaa.set_index('DATE')

In [8]:
#create flagging function to tag observations as good or bad based on rules:
    #if any of the following: 
        #observation has rain detected, temperature below 40 or above 95
            #day is flagged as bad (0)
def flagger(df):
    if (df['HOURLYPrecip'] > .05) or (df['HOURLYDRYBULBTEMPF'] < 40) or (df['HOURLYDRYBULBTEMPF'] > 95):
        return 0
    else:
        return 1
    
noaa['goodobs'] = noaa.apply(flagger, axis=1)

In [10]:
#for each station:
# 1. resample to 6 hour time periods with sum function, getting total good observations per 6 hour period
# 2. divide sum of positive responses per period by total number of observations per period
# 3. 6 hour periods which consist of more than 50% "good" observations are "good"
# 4. iterate through rows and check for 6 hour buffer: if "good" 6 hr period is preceded by another, it gets a 1 - else 0
stationlist = []
for num in noaa.groupby('STATION'):
    noaare = noaa[noaa['STATION'] == num[0]].resample('6H').sum()
    noaarecount = noaa[noaa['STATION'] == num[0]].resample('6H').size()
    noaare['sixhrprop'] = noaare['goodobs'] / noaarecount
    noaare['sixhourgood'] = [1 if x > 0.5 else 0 for x in noaare['sixhrprop']]
    sixhoursbuffgood = []
    for i in range(0, len(noaare)):
        if noaare.iloc[i].sixhourgood == 1 and noaare.iloc[i-1].sixhourgood == 1:
            sixhoursbuffgood.append(1)
        else:
            sixhoursbuffgood.append(0)    
    noaare['sixhoursbuffgood'] = sixhoursbuffgood    
    #namevar = num[0].replace(':','-')
    #noaare['sixhoursbuffgood'].to_csv('station'+namevar+'.csv')
    #noaare[['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE']] = noaa[noaa['STATION'] == num[0]][['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE']].reset_index().drop('DATE', axis=1)
    noaare['STATION'] = noaa[noaa['STATION'] == num[0]].reset_index().drop('DATE', axis=1)['STATION'][0]
    noaare['STATION_NAME'] = noaa[noaa['STATION'] == num[0]].reset_index().drop('DATE', axis=1)['STATION_NAME'][0]
    noaare['ELEVATION'] = noaa[noaa['STATION'] == num[0]].reset_index().drop('DATE', axis=1)['ELEVATION'][0]
    noaare['LATITUDE'] = noaa[noaa['STATION'] == num[0]].reset_index().drop('DATE', axis=1)['LATITUDE'][0]
    noaare['LONGITUDE'] = noaa[noaa['STATION'] == num[0]].reset_index().drop('DATE', axis=1)['LONGITUDE'][0]    
    stationlist.append(noaare)
    print("station", num[0], noaare.sixhoursbuffgood.value_counts())
out = pd.concat(stationlist)
out = out.drop(['HOURLYDRYBULBTEMPF', 'HOURLYPressureTendency', 'HOURLYPressureChange', 'HOURLYPrecip'], axis=1)
out.to_csv('noaa_out.csv')

    
    

station WBAN:00120 1    5127
0    1331
Name: sixhoursbuffgood, dtype: int64
station WBAN:00153 1    4536
0    2768
Name: sixhoursbuffgood, dtype: int64
station WBAN:00154 1    5208
0    2096
Name: sixhoursbuffgood, dtype: int64
station WBAN:00155 1    5582
0    1722
Name: sixhoursbuffgood, dtype: int64
station WBAN:00274 1    4389
0    2271
Name: sixhoursbuffgood, dtype: int64
station WBAN:00367 1    5228
0    2076
Name: sixhoursbuffgood, dtype: int64
station WBAN:00416 1    4591
0    1867
Name: sixhoursbuffgood, dtype: int64
station WBAN:03701 1    5859
0    1445
Name: sixhoursbuffgood, dtype: int64
station WBAN:03704 1    5473
0    1831
Name: sixhoursbuffgood, dtype: int64
station WBAN:03706 1    5460
0    1844
Name: sixhoursbuffgood, dtype: int64
station WBAN:03707 1    5588
0    1716
Name: sixhoursbuffgood, dtype: int64
station WBAN:03710 1    5113
0    2190
Name: sixhoursbuffgood, dtype: int64
station WBAN:03714 1    5597
0    1707
Name: sixhoursbuffgood, dtype: int64
station WBAN

TypeError: 'NoneType' object is not iterable